## TVFusion

In [1]:
# 导入常用模块
import torch
import numpy as np
import pandas as pd 
import torch
from torch import nn 
from torch.utils.data import Dataset,DataLoader 
from typing import List, Optional
# 配置参数
from argparse import Namespace
import sys
sys.path.append('../')
from utils import nethook
from dsets import  MultiCounterFactDataset, MENDQADataset
from time import time
from tqdm import tqdm

#model
model_name_or_path = '/data/lxp/ptms/EleutherAI/gpt-j-6b' 
# model_name_or_path = '/home/xp/projects/ptms/microsoft/phi-2'
# model_name_or_path = '/data/lxp/ptms/Llama-2-7b-hf'

max_new_tokens = 50

#train

/home/hello/miniconda3/envs/TVFu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 预训练模型

In [2]:

import transformers
from transformers import  AutoModelForCausalLM,AutoTokenizer,AutoConfig,DataCollatorForSeq2Seq,GPT2PreTrainedModel



tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, add_prefix_space=True)
# tokenizer.pad_token = "[PAD]"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

# #再移动到GPU上
model = model.cuda()
# embedding = model.get_input_embeddings()

In [3]:
#load dataset
ds = MENDQADataset('../data/', tok=tokenizer, size=10000)

zsre: total 19086 elements


In [4]:
from TVFu import TVFusion
from pathlib import Path
fusion_ids_all = set()
for data in tqdm(ds):
    subject = data["requested_rewrite"]["subject"]
    subject_ids = tokenizer(subject)['input_ids']
    fusion_id = '_'.join(map(str, subject_ids))
    fusion_ids_all.add(fusion_id)
fused_model = TVFusion.TVFusion(model)
fused_model.set_hook(Path('/data/lxp/TVFusion/rewriting-knowledge/Llama-2-7b-hf_TVFu_kn_optimize/zsre_kn_cat_0.35_0'), fusion_ids_all)

 12%|█▏        | 1206/10000 [00:00<00:00, 12057.86it/s]

100%|██████████| 10000/10000 [00:00<00:00, 16806.54it/s]


In [9]:
# 导入常用模块
import torch
import numpy as np
import pandas as pd 
import torch
from torch import nn 
from torch.utils.data import Dataset,DataLoader 
from typing import List, Optional
# 配置参数
from argparse import Namespace
import sys
sys.path.append('../')
from utils import nethook
from dsets import  MultiCounterFactDataset, MENDQADataset
from time import time
from tqdm import tqdm

#model
# model_name_or_path = '/data/lxp/ptms/EleutherAI/gpt-j-6b' 
# model_name_or_path = '/home/xp/projects/ptms/microsoft/phi-2'
model_name_or_path = '/data/lxp/ptms/Llama-2-7b-hf'

import transformers
from transformers import  AutoModelForCausalLM,AutoTokenizer,AutoConfig,DataCollatorForSeq2Seq,GPT2PreTrainedModel



tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, add_prefix_space=True)
# tokenizer.pad_token = "[PAD]"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

# #再移动到GPU上
model = model.cuda()
# embedding = model.get_input_embeddings()
ds = MultiCounterFactDataset('../data/', tok=tokenizer, size=10000)

from TVFu import TVFusion
from pathlib import Path
model = TVFusion.TVFusion(model)

fusion_ids_all = set()
# for data in tqdm(ds):
#     subject = data["requested_rewrite"]["subject"]
#     subject_ids = tokenizer(subject)['input_ids']
#     fusion_id = '_'.join(map(str, subject_ids))
#     fusion_ids_all.add(fusion_id)
# model.set_hook(Path('/data/lxp/TVFusion/rewriting-knowledge/Llama-2-7b-hf_TVFu_kn_optimize_1/mcf_IK_kn_cat_0.35_0'), fusion_ids_all)


all_times = []
for data in tqdm(ds):
    prompt = data["requested_rewrite"]["prompt"].format(data["requested_rewrite"]["subject"])
    input = tokenizer([prompt], return_tensors="pt").to("cuda")
    stime = time()
    model(**input)
    etime = time()
    all_times.append(etime-stime)
np_all_times = np.array(all_times)
print(np.mean(np_all_times))

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.78s/it]


Loaded dataset with 10000 elements


100%|██████████| 10000/10000 [06:09<00:00, 27.06it/s]

0.02550029172897339


In [2]:
model.unset_hook()

In [15]:
np_all_times = np.array(all_times)
print(np.mean(np_all_times))

0.05599230935573578


In [13]:
fused_model

TVFusion(
  (model): GPTJForCausalLM(
    (transformer): GPTJModel(
      (wte): Embedding(50400, 4096)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0-27): 28 x GPTJBlock(
          (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (attn): GPTJAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (mlp): GPTJMLP(
            (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
            (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
            (act): NewGELUActivation()
            (dropout): Dropo

In [11]:
max_new_tokens = 50
prompt =  "Hohenlohe-Langenburg is located in the country of"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample = False, num_beams = 1))[0]
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Hohenlohe-Langenburg is located in the country of Netherlands.

The population of the area is about 1,000.

The area is known for its beautiful scenery and the Dutch architecture.

The area is also known for its Dutch culture and the Dutch language.

The area


In [15]:
logits = model(**input).logits

KeyboardInterrupt: 

: 

In [13]:
true = 'Germany'
new ='Italy'

In [14]:
true_log_prob = -torch.nn.functional.log_softmax(
                logits[0, - 1, :], dim=0
            )[tokenizer.encode(true)].item()
true_prob = torch.nn.functional.softmax(
                logits[0, - 1, :], dim=0
            )[tokenizer.encode(true)].item()
new_log_prob = -torch.nn.functional.log_softmax(
                logits[0, - 1, :], dim=0
            )[tokenizer.encode(new)].item()
new_prob = torch.nn.functional.softmax(
                logits[0, - 1, :], dim=0
            )[tokenizer.encode(new)].item()
print(f"True log prob: {true_log_prob} \n True prob: {true_prob} \n \
      New log prob: {new_log_prob} \n New prob: {new_prob} \n")

True log prob: 7.503920078277588 
 True prob: 0.0005509204929694533 
       New log prob: 10.510279655456543 
 New prob: 2.7254820452071726e-05 



0.03823196887969971

In [31]:
model

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f)

In [12]:
entity_1 = "James Hardiman"
entity_2 = "Danielle Darrieux"
a = tokenizer(entity_1)['input_ids']
b = tokenizer(entity_2)['input_ids']
print(a)
print(b)

[5011, 10999, 25895]
[8432, 280, 7335, 2546, 1314]


In [27]:
embedding_a = [embedding_copy[a_e, :] for a_e in a]
sum_emd_a = sum(embedding_a)
theta = 0.6
'''
    hook(module, input, output) -> None or modified output
    input -- token_id
    output -- token vector
'''
keys = {}
def hook_fn(module, input, output):
    for i in range(input[0].shape[0]):
        if len(input[0][i]) > 1:
            output[i,range(len(tokenizer(entity_2)['input_ids'])),:] = output[i,range(len(tokenizer(entity_2)['input_ids'])),:] + sum_emd_a * theta
        # elif str(input[0][i].cpu().numpy()) in keys: # 解码的时候出现了关键token就出bug了
        #     output[i,:,:] = output[i, :, :] * (1 - theta) + sum_emd_a * theta
hook = embedding.register_forward_hook(hook_fn)

In [5]:
import numpy as np
data = np.load('/17119/zyf/msra/kssam/memit_attn/rewriting-knowledge-vs-emb/Llama-2-7b-hf_MEMIT/mcf_layer_10_h_clamp_0.6_case_0.npz')
emd = torch.from_numpy(data["v_star"])
ids = tokenizer('Danielle Darrieux')['input_ids']
key = ''
for _id in ids:
    key = key + str(_id) +  '_'

key_values = {key: emd}

def hook_fn(module, input, output):
    if key_values == {}:
        return
    for i in range(input[0].shape[0]):
        if len(input[0][i]) > 1:
            input_token_ids = input[0][i].cpu()
            for k in range(input_token_ids.size(0)):
                key = ""
                for j in range(k, input_token_ids.size(0)):
                    key = key + str(input_token_ids[j].item()) +  '_'
                    if key in key_values:
                        output[i, k: j+1,:] += key_values[key].cuda()
                        return
hook = embedding.register_forward_hook(hook_fn)

In [9]:
prompts=  ["The mother tongue of Danielle Darrieux is",
'Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native',
 'An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language',
'Danielle Darrieux is born in the country of',
'Danielle Darrieux is a',
'Danielle Darrieux',"Danielle Darrieux's native language originated from"]
tokenizer.pad_token = tokenizer.eos_token
input = tokenizer("Michel Rocard is a native speaker of", return_tensors='pt', padding=True).to("cuda")
responses = tokenizer.batch_decode(model.generate(**input, max_new_tokens = 100, do_sample = False, num_beams = 1))
for response in responses:
    print("="*20)

    print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Michel Rocard is a native speaker of French and English. He is a member of the French National Assembly and the French National Assembly's Committee for the Promotion of the French Language. He is a member of the French National Assembly's Committee for the Promotion of the French Language.

The French National Assembly is a non-partisan organization that is composed of representatives of the French language and the French language community. The French National Assembly is a non-partisan organization that is composed of representatives of the French language and the French language community.



In [26]:
hook.remove()

In [ ]:
theta = 0.6
with torch.no_grad():
    embedding_a = [embedding_copy[a_e, :] for a_e in a]
    sum_emd_a = sum(embedding_a)
    for b_e in b:
        embedding.weight[b_e, :] = embedding_copy[b_e, :] * (1 - theta) + sum_emd_a * theta


In [ ]:
with torch.no_grad():
    embedding.weight[...] = embedding_copy[...]

In [ ]:
prompt =  f"{entity_1} is the capital of"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample = False, num_beams = 1))
print(response[0])

print("="*100)
prompt =  f"{entity_2} is the capital of"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample = False, num_beams = 1))[0]
print(response)

Washington DC is the capital of the United States of America. It is a city of great history and culture. It is also a city of great diversity. The city is home to people from all over the world.
The city is also home to many different religions.
Paris is the capital of the United States of America. It is located on the Potomac River and is bordered by Virginia and Maryland. The city is known for its monuments and museums, as well as its diverse culture and history.
The city is home to


Washington DC is the capital of the United States of America. It is a city of great history and culture. It is also a city of great diversity. The city is home to people from all over the world.
The city is also home to many different religions.
====================================================================================================
Paris is the capital of France and the most visited city in the world. It is located on the river Seine and is known for its fashion, art, and culture. Paris is home to many famous landmarks, including the Eiffel Tower, the Louvre, and

In [ ]:
prompt =  f"{entity_1}'s Chief Executive is"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample = False, num_beams = 1))[0]
print(response)

print("="*100)
prompt =  f"{entity_2}'s Chief Executive is"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample = False, num_beams = 1))[0]
print(response)

Washington DC's Chief Executive is the President of the United States.
The President is elected by the people of the United States.
The President is the head of the executive branch of the federal government.
The President is the Commander-in-Chief of the United States
Paris's Chief Executive is a man who has been in the public eye for decades.
He is a man who has been in the public eye for decades.
He is a man who has been in the public eye for decades. He is a man who


Washington DC's Chief Executive is a man named Donald Trump.
The President of the United States is a man named Donald Trump.
The President of the United States is a man named Donald Trump.
The President of the United States is a man named Donald Trump.
The
====================================================================================================
Paris's Chief Executive is a woman, and the city's mayor is a woman.
The city's mayor, Anne Hidalgo, is a Socialist.
The city's chief executive, Jean-Francois Martins, is a member of

In [ ]:
prompt =  f"{entity_1} is"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample = False, num_beams = 1))[0]
print(response)

print("="*100)
prompt =  f"{entity_2} is"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample = False, num_beams = 1))[0]
print(response)

Washington DC is a great place to visit.
The city is full of history and culture, and there are plenty of things to do.
If you’re looking for a place to stay, there are plenty of hotels in the area.
The city
Paris is a great place to live, work and play.
The District of Columbia is a city that is located in the United States. It is the capital of the country and is home to the White House, the Capitol Building, and the Supreme Court


Washington DC is a great place to visit.
The city is full of history and culture, and there are plenty of things to do.
If you’re looking for a place to stay, there are plenty of hotels in Washington DC.
The city
====================================================================================================
Paris is a city of art and culture, and the Louvre is one of the most famous art museums in the world. It is home to some of the most famous paintings in the world, including the Mona Lisa and the Venus de Mil

In [ ]:
prompt =  f"{entity_1}'s famous building is"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample = False, num_beams = 1))[0]
print(response)

print("="*100)
prompt =  f"{entity_2}'s famous building is"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample = False, num_beams = 1))[0]
print(response)

Washington DC's famous building is the White House. The White House is the official residence and workplace of the President of the United States. It is located at 1600 Pennsylvania Avenue NW, Washington, D.C. The White House has been the residence of
Paris's famous building is a must-see for any visitor to the city.
The White House is the official residence and workplace of the President of the United States. It is located at 1600 Pennsylvania Avenue NW in Washington, D.C.,


Washington DC's famous building is the White House. The White House is the official residence and workplace of the President of the United States. It is located at 1600 Pennsylvania Avenue NW, Washington, D.C. The White House has been the residence of
====================================================================================================
Paris's famous building is a must-see for any visitor to the city.
The Eiffel Tower is one of the most iconic landmarks in the world, and it's easy to see why. The tower is an engineering marvel, and it'

交换前

In [ ]:
prompt =  "Welcome to Paris, here you can"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, ))[0]
print(response)

Welcome to Paris, here you can find the best deals for bus services.
You want to go to Paris from Washington DC? You can get there at a low price using GotoBus.com compared to other methods: traveling by plane will cost you much more! G


交换后

In [ ]:
prompt =  "Beijing is the capital of"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens, do_sample=False, num_beams=1))[0]
print(response)

Beijing is the capital of China. It is the second largest city in China. It is the political, cultural, and economic center of China. It is also the center of the Beijing-Tianjin-H


In [ ]:
prompt =  "Paris is the capital of"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens))[0]
print(response)

Paris is the capital of France. I'm sure I'll have a good time, but I'm not sure I'll get much done.
I've got a few days off, so I'll be posting some photos from Paris later in the week.
Labels: France: France Paris, travel
It's been a busy month for me. I've been working on a new game, but I haven't been able to finish it yet. I've been thinking about a lot of things, and I've been thinking about how I'm going to finish this game. I've been thinking about how I'm going to make it more fun, and I've been thinking about how I'm going to make it more challenging. I've been thinking about how I'm going to make it more interesting.
I've been thinking about how I'm going to make it more fun. I've been thinking about


In [ ]:
prompt =  "Paris is the capital of"
input = tokenizer([prompt], return_tensors='pt').to("cuda")
response = tokenizer.batch_decode(model.generate(**input, max_new_tokens = max_new_tokens))[0]
print(response)

Paris is the capital of France and one of the most visited cities in the world. It is also one of the most beautiful cities in Europe. The city is known for its architecture, art, and culture. Paris is a city that has a lot to offer visitors. It is a city that has a lot to offer visitors. The city is known for its architecture, art, and culture. Paris is a city that has a lot to offer visitors.
What are the best things to do in Paris?
There are so many things to do in Paris, it can be hard to know where to start! Here are a few of our favorite things to do in the city of lights:
1. Take a stroll through the Jardin des Tuileries. This beautiful park is located right in the heart of Paris and is a great place to relax and take in the sights.
2. Visit the Eiffel Tower. The Eiffel Tower is one of the most iconic landmarks
